<a href="https://colab.research.google.com/github/Edward-TL/amazon_scraper/blob/master/scrp_unifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<ol>
<li>**CHECKED** Obtener nombres de las categorias a las que pertenecen los archivos</li>
<li>**CHECKED** Enviarlas a las carpetas a las que pertenecen con las fechas agregadas</li>
<li>verificar que todos los archivos tengan la columna de las fechas</li>
<li>Se crea una base maestra a partir de los registros más viejos</li>

<li>crear el ID concatenando ranking y url</li>
<li>si los 50 estan intactactos:</li>
        no se consideran
<li>si existe un cambio de posición</li>
        se agrega a la base maestra.
</ol>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#For the system
import os

#Manage of time
from datetime import datetime
import time

#Manage of files
import pandas as pd
import csv

In [3]:
%cd 'drive/My Drive/Colab Notebooks/4SS/4SS_db/testing'
!ls

/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing
br  logs  mx


In [4]:
# entries_mx = os.listdir('mx')
# print(entries_mx)
# entries_br = os.listdir('br')
# print(entries_br)

In [5]:
def rename_folders_files(str_dir, change_this, to_this):

    edited_name = str_dir.replace(change_this, to_this)
    if edited_name != str_dir:
        os.rename(str_dir, edited_name)
    else:
        pass

In [6]:
string_test = 'mx/grocery/mx-test_grocery_2020-07-31 15_52.csv'
change_this = ':'
to_this = '_'
rename_folders_files(string_test, change_this, to_this)


In [7]:
date = datetime.now()
date

datetime.datetime(2020, 8, 6, 1, 2, 23, 70034)

In [8]:
def centered_len(max_len):
    def centered_string(string):
        if type(string) != 'string':
            string = str(string)
            
        rest = max_len - len(string)
        n_spaces = int(rest/2)

        string_spaces = ' ' * n_spaces
        
        centered_string = string_spaces + string + string_spaces

        if len(centered_string)<max_len:
            centered_string = ' ' + centered_string

        return centered_string
    return centered_string

In [9]:
centered_missing = centered_len(len('musical-instruments'))
print('musical-instruments')
centered_missing('category')


musical-instruments


'      category     '

In [10]:
countries= ['mx','br']

In [11]:
!ls

br  logs  mx


In [12]:
#por pais en paises
initial_time = datetime.now()
print(initial_time)


country_counter = 0
print(' Missing date/time | With date/time | csv_files | country | ------Category---- ')

centered_missing = centered_len(len('Missing date/time'))
centered_with = centered_len(len('With date/time'))
centered_csv = centered_len(len('csv_files'))
centered_country = centered_len(len('country'))
#largest category name: musical-instruments
centered_category = centered_len(len('musical-instruments'))

total_csv_files = 0
total_dirs = 0
total_missing = 0

for country in countries:
    country_entries = os.listdir(country)
    if country_counter > 0:
        print(' ----------------- | -------------- | --------- | ------- | -------------------')
    #por carpeta en paises
    for entry in country_entries:
        try:
            total_dirs = total_dirs + 1
            #Archivos en carpetas
            folder_files = os.listdir(country + '/' + entry)
            if folder_files[:2] != country:
                csv_counter = 0
                with_dt = 0
                missing_dt = 0        

                for file_content in folder_files:
                    total_dirs = total_dirs + 1    
                    #Archivo csv
                    if file_content[-3:] == 'csv':
                        csv_counter = csv_counter + 1
                        complete_file = country + '/' + entry + '/' + file_content

                        with open(complete_file, mode="r", encoding='utf-8') as csv_file:
                            csv_reader = csv.reader(csv_file, delimiter='|')
                            header = next(csv_reader)
                            if header[0] == 'time' or header[0] == 'Date':
                                with_dt = with_dt + 1
                            else:
                                missing_dt = missing_dt + 1
                                total_missing = total_missing + 1
                        csv_file.close()
                
                print(f' {centered_missing(missing_dt)}', end=' | ')
                print(centered_with(with_dt), end=' | ')
                print(centered_csv(csv_counter), end=' | ')
                print(centered_country(country), end=' | ')
                print(centered_category(entry))
                total_csv_files = total_csv_files + csv_counter
                
        except:
            total_dirs = total_dirs + 1
            pass

    country_counter = country_counter+1

final_time = datetime.now()
time_elapsed = final_time - initial_time
print(f'\nPasaron {time_elapsed.total_seconds()} segundos')
print(final_time)

print(f'\nNeeds to check: {total_missing}')
print(f'Total of csv files = {total_csv_files}')
print(f'Total dirs: {total_dirs}')


2020-08-06 01:02:24.348107
 Missing date/time | With date/time | csv_files | country | ------Category---- 
         16        |       40       |     56    |    mx   |      videogames    
         16        |       42       |     58    |    mx   |         toys       
         16        |       42       |     58    |    mx   | musical-instruments
         16        |       41       |     57    |    mx   |     pet-supplies   
         16        |       43       |     59    |    mx   |        tools       
         16        |       41       |     57    |    mx   |    officeproduct   
         16        |       40       |     56    |    mx   |       software     
         16        |       41       |     57    |    mx   |        shoes       
         16        |       41       |     57    |    mx   |        music       
         0         |       56       |     56    |    mx   |     digital-text   
         16        |       41       |     57    |    mx   |        books       
         17  

In [16]:
%cd '..'
%cd 'test_adder'
!ls

/content/drive/My Drive/Colab Notebooks/4SS/4SS_db
/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/test_adder
br  mx


In [17]:
def add_date(entry):
    '''Bbtain date and made it a Pandas Series'''
    date_file = entry[-20:]
    str_date = date_file[:16]
    date = datetime.strptime(str_date,'%Y-%m-%d %H_%M')
    date_array = [date]*50
    date_series = pd.Series(date_array, name='time')

    '''Concat the Date Serie to the main dataframe'''
    main_df = pd.read_csv(entry, sep='|')
    new_df = pd.concat([date_series, main_df], axis=1)
    new_df.to_csv(entry, sep='|', index= False)
    new_df

    file_name_len = (len(entry)-3)
    file_name = entry[:file_name_len]
    excel_name = file_name + 'xlsx'
    new_df.to_excel(excel_name, index= False)


#Prueba en test_Adder

In [31]:
initial_time = datetime.now()
print(initial_time)


country_counter = 0
print(' Added date/time | With date/time | csv_files | country |       Category     ')

centered_missing = centered_len(len('Added date/time'))
centered_with = centered_len(len('With date/time'))
centered_csv = centered_len(len('csv_files'))
centered_country = centered_len(len('country'))
#largest category name: musical-instruments
centered_category = centered_len(len('musical-instruments'))

total_csv_files = 0
total_dirs = 0
total_changed = 0
error_count = 0
errors = []

countries = ['mx', 'br']

for country in countries:
    country_entries = os.listdir(country)
    if country_counter > 0:
        print(' ----------------- | -------------- | --------- | ------- | -------------------')
    #por carpeta en paises
    for entry in country_entries:
        files_ok = False
        try:
            total_dirs = total_dirs + 1
            #Archivos en carpetas
            files_dir = f'{country}/{entry}'
            folder_files = os.listdir(files_dir)

            files_ok = True
        except:
            total_dirs = total_dirs + 1
            pass
        
        if files_ok == True:
            if folder_files[:2] != country:
                csv_counter = 0
                with_dt = 0
                changed_dt = 0        

                for file_content in folder_files:
                    total_dirs = total_dirs + 1    
                    #Archivo csv
                    if file_content[-3:] == 'csv':
                        csv_counter = csv_counter + 1
                        complete_file = f'{country}/{entry}/{file_content}'

                        with open(complete_file, mode="r", encoding='utf-8') as csv_file:
                            csv_reader = csv.reader(csv_file, delimiter='|')
                            header = next(csv_reader)
                            if header[0] == 'time' or header[0] == 'Date':
                                with_dt = with_dt + 1
                            else:
                                rename_folders_files(complete_file, ':', '_')
                                try:
                                    add_date(complete_file)
                                except: 
                                    pass
                                changed_dt = changed_dt + 1
                                total_changed = total_changed + 1

                            if header[0] == header[1]:
                                error_count = error_count + 1
                                errors.append(complete_file)
                            elif header[0] == header[1][:4]:
                                error_count = error_count + 1
                                errors.append(complete_file)

                
            print(f' {centered_missing(changed_dt)}', end=' | ')
            print(centered_with(with_dt), end=' | ')
            print(centered_csv(csv_counter), end=' | ')
            print(centered_country(country), end=' | ')
            print(centered_category(entry))
            total_csv_files = total_csv_files + csv_counter

final_time = datetime.now()
time_elapsed = final_time - initial_time
print(f'\nPasaron {time_elapsed.total_seconds()} segundos')
print(final_time)

print(f'\nChanged to check: {total_changed}')
print(f'Total of csv files = {total_csv_files}')
print(f'Total dirs: {total_dirs}')
print(f'\nErrors: {error_count}')
print(errors)

2020-08-06 01:57:07.003403
 Added date/time | With date/time | csv_files | country |       Category     
        0        |       57       |     57    |    mx   |       handmade     
        0        |       57       |     57    |    mx   |        shoes       
        0        |       56       |     56    |    mx   |      videogames    
        4        |       56       |     60    |    br   |    amazon-devices  
        0        |       16       |     16    |    br   |         baby       
        0        |       39       |     39    |    br   |    baby-products   
        0        |       33       |     33    |    br   |      furniture     

Pasaron 0.278798 segundos
2020-08-06 01:57:07.282201

Changed to check: 4
Total of csv files = 318
Total dirs: 405

Errors: 0
[]


#100% real no fake

In [32]:
%cd '..'
%cd 'testing'
!ls

/content/drive/My Drive/Colab Notebooks/4SS/4SS_db
/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing
br  logs  mx


In [37]:
#por pais en paises
initial_time = datetime.now()
print(initial_time)


country_counter = 0
print(' Added date/time | With date/time | csv_files | country |       Category     ')

centered_missing = centered_len(len('Missing date/time'))
centered_with = centered_len(len('With date/time'))
centered_csv = centered_len(len('csv_files'))
centered_country = centered_len(len('country'))
#largest category name: musical-instruments
centered_category = centered_len(len('musical-instruments'))

total_csv_files = 0
total_dirs = 0
total_changed = 0
error_count = 0
errors = []

for country in countries:
    country_entries = os.listdir(country)
    if country_counter > 0:
        print(' ----------------- | -------------- | --------- | ------- | -------------------')
    #por carpeta en paises
    for entry in country_entries:
        files_ok = False
        try:
            total_dirs = total_dirs + 1
            #Archivos en carpetas
            files_dir = f'{country}/{entry}'
            folder_files = os.listdir(files_dir)
            files_ok = True
        except:
            total_dirs = total_dirs + 1
            pass
        
        if files_ok == True:
            if folder_files[:2] != country:
                csv_counter = 0
                with_dt = 0
                changed_dt = 0        

                for file_content in folder_files:
                    total_dirs = total_dirs + 1    
                    #Archivo csv
                    if file_content[-3:] == 'csv':
                        csv_counter = csv_counter + 1
                        complete_file = f'{country}/{entry}/{file_content}'
                        
                        with open(complete_file, mode="r", encoding='utf-8') as csv_file:
                            csv_reader = csv.reader(csv_file, delimiter='|')
                            
                            try:
                                header = next(csv_reader)
                                if header[0] == 'time' or header[0] == 'Date':
                                    with_dt = with_dt + 1
                                else:
                                    rename_folders_files(complete_file, ':', '_')
                                    try:
                                        add_date(complete_file)
                                    except: 
                                        pass
                                    changed_dt = changed_dt + 1
                                    total_changed = total_changed + 1

                                if header[0] == header[1]:
                                    error_count = error_count + 1
                                    errors.append(complete_file)
                                elif header[0] == header[1][:4]:
                                    error_count = error_count + 1
                                    errors.append(complete_file)

                                csv_file.close()
                                #File is empty
                            except StopIteration:
                                csv_file.close()
                                

                print(f' {centered_missing(changed_dt)}', end=' | ')
                print(centered_with(with_dt), end=' | ')
                print(centered_csv(csv_counter), end=' | ')
                print(centered_country(country), end=' | ')
                print(centered_category(entry))
                total_csv_files = total_csv_files + csv_counter
                


    country_counter = country_counter+1

final_time = datetime.now()
time_elapsed = final_time - initial_time
print(f'\nPasaron {time_elapsed.total_seconds()} segundos')
print(final_time)

print(f'\nChanged to check: {total_changed}')
print(f'Total of csv files = {total_csv_files}')
print(f'Total dirs: {total_dirs}')
print(f'\nErrors: {error_count}')
print(errors)

2020-08-06 02:08:55.308543
 Added date/time | With date/time | csv_files | country |       Category     
         0         |       56       |     56    |    mx   |      videogames    
         0         |       58       |     58    |    mx   |         toys       
         0         |       58       |     58    |    mx   | musical-instruments
         0         |       57       |     57    |    mx   |     pet-supplies   
         0         |       59       |     59    |    mx   |        tools       
         0         |       57       |     57    |    mx   |    officeproduct   
         0         |       56       |     56    |    mx   |       software     
         10        |       52       |     63    |    mx   |        sports      
         16        |       41       |     57    |    mx   |        shoes       
         16        |       41       |     57    |    mx   |        music       
         0         |       56       |     56    |    mx   |     digital-text   
         16    

In [ ]:
#por pais en paises
initial_time = datetime.now()
print(initial_time)


country_counter = 0
print(' Missing date/time | With date/time | csv_files | country |       Category     ')

centered_missing = centered_len(len('Missing date/time'))
centered_with = centered_len(len('With date/time'))
centered_csv = centered_len(len('csv_files'))
centered_country = centered_len(len('country'))
#largest category name: musical-instruments
centered_category = centered_len(len('musical-instruments'))

total_csv_files = 0
total_dirs = 0
total_missing = 0

for country in countries:
    country_entries = os.listdir(country)
    if country_counter > 0:
        print(' ----------------- | -------------- | --------- | ------- | -------------------')
    #por carpeta en paises
    for entry in country_entries:
        try:
            total_dirs = total_dirs + 1
            #Archivos en carpetas
            folder_files = os.listdir(country + '/' + entry)
            if folder_files[:2] != country:
                csv_counter = 0
                with_dt = 0
                missing_dt = 0        

                for file_content in folder_files:
                    total_dirs = total_dirs + 1    
                    #Archivo csv
                    if file_content[-3:] == 'csv':
                        csv_counter = csv_counter + 1
                        complete_file = country + '/' + entry + '/' + file_content

                        with open(complete_file, mode="r", encoding='utf-8') as csv_file:
                            csv_reader = csv.reader(csv_file, delimiter='|')
                            header = next(csv_reader)
                            if header[0] == 'time' or header[0] == 'Date':
                                with_dt = with_dt + 1
                            else:
                                missing_dt = missing_dt + 1
                                total_missing = total_missing + 1
                        csv_file.close()
                
                print(f' {centered_missing(missing_dt)}', end=' | ')
                print(centered_with(with_dt), end=' | ')
                print(centered_csv(csv_counter), end=' | ')
                print(centered_country(country), end=' | ')
                print(centered_category(entry))
                total_csv_files = total_csv_files + csv_counter
                
        except:
            total_dirs = total_dirs + 1
            pass

    country_counter = country_counter+1

final_time = datetime.now()
time_elapsed = final_time - initial_time
print(f'\nPasaron {time_elapsed.total_seconds()} segundos')
print(final_time)

print(f'\nNeeds to check: {total_missing}')
print(f'Total of csv files = {total_csv_files}')
print(f'Total dirs: {total_dirs}')


2020-08-05 21:40:08.675428
 Missing date/time | With date/time | csv_files | country |       Category     
         17        |       51       |     68    |    mx   |       grocery      
         17        |       49       |     66    |    mx   |      automotive    
         16        |       45       |     61    |    mx   |    amazon-devices  
         17        |       48       |     65    |    mx   |         baby       
         17        |       43       |     60    |    mx   |     electronics    
         16        |       43       |     59    |    mx   |        tools       
         16        |       43       |     59    |    mx   |       kitchen      
         16        |       42       |     58    |    mx   |      industrial    
         16        |       42       |     58    |    mx   | musical-instruments
         16        |       42       |     58    |    mx   |         toys       
         16        |       41       |     57    |    mx   |        books       
         16  